# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [15]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [16]:
!cat talks.csv

title,type,url_slug,venue,date,location,talk_url,description
"From low-code geometric algebra to no-code geometric 
deep learning: computational models, simulation algorithms and authoring platforms for immersive scientific visualization, experiential visual analytics and the upcoming educational metaverse",Keynote Talk,keynote-17,IEEE Virtual Reality 2023,2023-05-03,online,https://papagiannakis.github.io/files/GP-IEEEVRkeynote2023.pdf,"More than 1 billion jobs, almost one-third of all jobs worldwide, are likely to be transformed by technology in the next decade, according to OECD and World Economic Forum estimates. In addition, 5 billion people today lack access to proper surgical and anesthesia care, due to the limited number of health professionals entering the workforce, as a direct result of the lack of innovation in medical training over the last 150 years. 
This growing need for continuous upskill and reskill, becomes even more critical in the post COVID-19 pandemic era. Extende

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [17]:
#talks = pd.read_csv("talks.csv", sep="\t", header=0)
talks = pd.read_csv("talks.csv", sep=",", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,From low-code geometric algebra to no-code geo...,Keynote Talk,keynote-17,IEEE Virtual Reality 2023,2023-05-03,online,https://papagiannakis.github.io/files/GP-IEEEV...,"More than 1 billion jobs, almost one-third of ..."
1,Medical Spatial Computing: Unraveling the Futu...,Invited Talk,talk-31,HarvardXR Conference 2023,2023-04-15,"Harvard University, Boston MA",https://papagiannakis.github.io/files/GP-Harva...,The main focus of this talk is providing an ov...
2,The 6Gs of educational XR spatial computing ap...,Invited Talk,talk-33,Harvard Mass General Brigham Extended Reality ...,2023-07-19,online,https://papagiannakis.github.io/files/GP-MGBXR...,Extended Reality (XR) together with spatial co...
3,Grand challenges for XR computational medicine...,Invited Talk,talk-27,"Laboratory of Cognitive Neuroscience, EPFL",2021-06-15,"Campus Biotech, Geneva, Switzerland",NaN,Invited talk by Dr. Bruno Herbelin and Prof. O...
4,XR simulation-based empirical learning for vir...,Invited Talk,talk-22,World XR Forum 2019,2019-07-10,"Crans Montana, Switzerland",NaN,NaN
5,Massive Open Online Courseware in Virtual Reality,Invited Talk,talk-21,Coursera HQ,2019-07-02,"Mountain View, CA",NaN,NaN
6,"First coop,networked VR collaborative shared s...",Invited Talk,talk-20,Stanford University Medical School,2018-06-25,"Palo Alto, CA",https://www.youtube.com/watch?v=jnulGrlLXbA&t=22s,"First collaborative, networked VR medical trai..."
7,From Geometric Algebra to Psychomotor Surgical...,Invited Talk,talk-15,"University of Southern California, Institute o...",2018-06-22,"Los Angeles, CA",NaN,Invited Talk by Prof. Ari Shapiro at USC-ICT
8,Gamified Presence in virtual museums,Tutorial,talk-13,CGI2017 Conferece,2017-06-20,"Yokohama, Japan",NaN,NaN
9,Mixed Reality and Gamification for Cultural He...,Tutorial,talk-12,"Politecnico di Milano, Department of Architect...",2016-07-15,"Milan, Italy",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [18]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [19]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [20]:
!ls ../_talks

2015-11-15-talk-10.md    2021-06-15-talk-27.md    2023-09-22-talk-21.md
2016-07-15-talk-12.md    2022-06-20-keynote-15.md 2023-10-12-talk-22.md
2017-06-20-talk-13.md    2023-04-15-talk-31.md    2023-11-20-talk-23.md
2018-06-22-talk-15.md    2023-05-03-keynote-17.md 2024-08-01-talk-24.md
2018-06-25-talk-20.md    2023-07-19-talk-33.md    2024-11-15-keynote-25.md
2019-07-02-talk-21.md    2023-08-06-talk-19.md    talkmap.ipynb
2019-07-10-talk-22.md    2023-08-07-talk-20.md


In [21]:
!cat ../_talks/2023.05.03-keynote-17.md

cat: ../_talks/2023.05.03-keynote-17.md: No such file or directory
